So far we have constructed our dataframe for analysis, therefore this part of project is to apply dataframe

### Instruction
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together.
- once you are happy with your analysis, submit a link to the new Notebook on your Github repository.

#### In this notebook, I will analyze the toronto location data similar to what we did in the lab on the New York city location data

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
nyc_df = newyork_data['features']

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
# instantiate the dataframe
df = pd.DataFrame(columns=column_names)

In [5]:
for data in nyc_df:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]

    df = df.append({'Borough': borough,
                    'Neighborhood': neighborhood_name,
                    'Latitude': neighborhood_lat,
                    'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### 0. Processing the data, we will onty use borough to do the analysis

#### Use geopy library to get the latitude and longitude values of Toronto City.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent to_explorer, as shown below.

In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [9]:
df.head(20)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [10]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10.25)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)     
map_newyork

### 1. Explore the neighborhoods of *NYC*
#### Let's create a function to repeat the same process to all the neighborhoods in *NYC*

In [11]:
CLIENT_ID='TLGC4FJNI3HPGNHXVFD0MD3DLUA2EEB01V5FZYDVDQOJXVRA'
CLIENT_SECRET='EPFIIO0TJ23CU0KZKXJVEFBL4AN4INFUPMK0NZZQ1NZSWL4S' 
VERSION='20191121'
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called ny_venues.

In [12]:
ny_venues = getNearbyVenues(names=df['Neighborhood'],
                            latitudes=df['Latitude'],
                            longitudes=df['Longitude'])

#### Let's check the size of the resulting dataframe

In [13]:
print(ny_venues.shape)
ny_venues.head()

(10290, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


#### Let's check how many venues were returned for each neighborhood

In [14]:
ny_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,31,31,31,31,31,31
Annadale,14,14,14,14,14,14
Arden Heights,5,5,5,5,5,5
Arlington,6,6,6,6,6,6
Arrochar,19,19,19,19,19,19


#### Let's find out how many unique categories can be curated from all the returned venues

In [15]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 429 uniques categories.


### 2. Analyze Each Neighborhood

In [16]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]
ny_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-

#### And let's examine the new dataframe size.

In [17]:
ny_onehot.shape

(10290, 429)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [18]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-

#### Let's confirm the new size

In [19]:
ny_grouped.shape

(301, 429)

#### Let's print each neighborhood along with the top 5 most common venues

In [20]:
num_top_venues = 5
for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                venue  freq
0         Supermarket  0.10
1         Pizza Place  0.10
2       Deli / Bodega  0.06
3         Bus Station  0.06
4  Chinese Restaurant  0.06


----Annadale----
                 venue  freq
0  American Restaurant  0.21
1          Pizza Place  0.14
2                  Pub  0.07
3               Bakery  0.07
4         Dance Studio  0.07


----Arden Heights----
          venue  freq
0      Pharmacy   0.2
1      Bus Stop   0.2
2   Coffee Shop   0.2
3  Home Service   0.2
4   Pizza Place   0.2


----Arlington----
                 venue  freq
0        Boat or Ferry  0.17
1  American Restaurant  0.17
2        Grocery Store  0.17
3         Intersection  0.17
4             Bus Stop  0.17


----Arrochar----
                   venue  freq
0               Bus Stop  0.16
1     Italian Restaurant  0.11
2          Deli / Bodega  0.11
3                  Hotel  0.05
4  Outdoors & Recreation  0.05


----Arverne----
             venue  freq
0        Surf Spot  0.22

                venue  freq
0       Deli / Bodega  0.15
1  Spanish Restaurant  0.08
2  Italian Restaurant  0.08
3            Pharmacy  0.04
4              Bakery  0.04


----Concourse Village----
                  venue  freq
0         Deli / Bodega  0.12
1  Fast Food Restaurant  0.09
2           Supermarket  0.06
3   Sporting Goods Shop  0.06
4        Sandwich Place  0.06


----Coney Island----
                          venue  freq
0              Baseball Stadium  0.12
1          Caribbean Restaurant  0.12
2                  Skating Rink  0.06
3                    Food Stand  0.06
4  Theme Park Ride / Attraction  0.06


----Corona----
                venue  freq
0  Mexican Restaurant  0.11
1      Sandwich Place  0.11
2   Convenience Store  0.06
3    Basketball Court  0.06
4  Chinese Restaurant  0.06


----Country Club----
               venue  freq
0     Sandwich Place   0.4
1         Playground   0.2
2  Mobile Phone Shop   0.2
3                Spa   0.2
4        Yoga Studio   0.0


-

4  Italian Restaurant  0.04


----Graniteville----
                 venue  freq
0          Supermarket   1.0
1          Yoga Studio   0.0
2    Outdoor Sculpture   0.0
3  Peruvian Restaurant   0.0
4   Persian Restaurant   0.0


----Grant City----
               venue  freq
0           Bus Stop  0.08
1        Pizza Place  0.08
2  Convenience Store  0.08
3      Grocery Store  0.04
4  Food & Drink Shop  0.04


----Grasmere----
                           venue  freq
0                       Bus Stop  0.17
1                           Bank  0.09
2                     Bagel Shop  0.09
3             Italian Restaurant  0.04
4  Vegetarian / Vegan Restaurant  0.04


----Gravesend----
                venue  freq
0  Italian Restaurant  0.14
1         Pizza Place  0.10
2              Bakery  0.10
3       Metro Station  0.07
4              Lounge  0.07


----Great Kills----
                venue  freq
0         Pizza Place  0.14
1  Italian Restaurant  0.10
2                 Bar  0.10
3   Food & Drink 

4  Latin American Restaurant  0.17


----Lower East Side----
                venue  freq
0                Café  0.05
1  Chinese Restaurant  0.05
2         Coffee Shop  0.05
3         Art Gallery  0.05
4         Pizza Place  0.05


----Madison----
                venue  freq
0          Bagel Shop  0.22
1  Italian Restaurant  0.11
2    Insurance Office  0.11
3         Pizza Place  0.11
4      Pilates Studio  0.11


----Malba----
                   venue  freq
0              Rest Area   0.5
1           Tennis Court   0.5
2            Yoga Studio   0.0
3  Outdoors & Recreation   0.0
4    Peruvian Restaurant   0.0


----Manhattan Beach----
            venue  freq
0            Café  0.18
1        Bus Stop  0.18
2            Food  0.09
3  Sandwich Place  0.09
4     Pizza Place  0.09


----Manhattan Terrace----
                venue  freq
0         Pizza Place  0.13
1          Donut Shop  0.13
2  Chinese Restaurant  0.09
3       Deli / Bodega  0.04
4   Convenience Store  0.04


----Manhattan V

4     Convenience Store  0.08


----Ozone Park----
         venue  freq
0  Pizza Place  0.09
1     Pharmacy  0.09
2        Diner  0.06
3         Bank  0.06
4          Gym  0.06


----Paerdegat Basin----
              venue  freq
0   Harbor / Marina   0.2
1     Moving Target   0.2
2              Food   0.2
3       Auto Garage   0.2
4  Asian Restaurant   0.2


----Park Hill----
                  venue  freq
0              Bus Stop  0.33
1           Coffee Shop  0.17
2  Gym / Fitness Center  0.17
3                 Hotel  0.17
4    Athletics & Sports  0.17


----Park Slope----
                 venue  freq
0          Coffee Shop  0.06
1  American Restaurant  0.06
2         Burger Joint  0.06
3   Italian Restaurant  0.04
4          Pizza Place  0.04


----Parkchester----
                venue  freq
0         Pizza Place  0.11
1         Supermarket  0.11
2          Kids Store  0.06
3                Bank  0.06
4  Chinese Restaurant  0.06


----Pelham Bay----
                       venue  freq


            venue  freq
0  Clothing Store  0.09
1        Boutique  0.08
2     Art Gallery  0.05
3   Women's Store  0.04
4      Shoe Store  0.04


----Somerville----
                 venue  freq
0                 Park   1.0
1          Yoga Studio   0.0
2            Pet Store   0.0
3             Pet Café   0.0
4  Peruvian Restaurant   0.0


----Soundview----
                 venue  freq
0   Chinese Restaurant  0.18
1         Liquor Store  0.12
2         Burger Joint  0.06
3  Fried Chicken Joint  0.06
4          Video Store  0.06


----South Beach----
                venue  freq
0                Pier   0.4
1       Deli / Bodega   0.2
2               Beach   0.2
3  Athletics & Sports   0.2
4         Yoga Studio   0.0


----South Jamaica----
           venue  freq
0    Bus Station  0.29
1  Grocery Store  0.14
2         Bakery  0.14
3       Bus Line  0.14
4    Supermarket  0.14


----South Ozone Park----
                  venue  freq
0                  Park  0.27
1         Deli / Bodega  0.1

4   Fried Chicken Joint  0.05


----Woodhaven----
                venue  freq
0       Deli / Bodega  0.13
1            Pharmacy  0.09
2                Bank  0.09
3         Video Store  0.04
4  Chinese Restaurant  0.04


----Woodlawn----
               venue  freq
0      Deli / Bodega  0.16
1                Pub  0.08
2         Playground  0.08
3        Pizza Place  0.08
4  Food & Drink Shop  0.08


----Woodrow----
                venue  freq
0            Pharmacy  0.11
1  Italian Restaurant  0.05
2          Donut Shop  0.05
3                Bank  0.05
4    Sushi Restaurant  0.05


----Woodside----
                       venue  freq
0            Thai Restaurant  0.06
1              Grocery Store  0.06
2                     Bakery  0.05
3        Filipino Restaurant  0.05
4  Latin American Restaurant  0.05


----Yorkville----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2                 Gym  0.06
3                 Bar  0.06
4         Pizza Place  0.0

#### Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Chinese Restaurant,Bus Station,Deli / Bodega,Fried Chicken Joint,Breakfast Spot,Grocery Store,Martial Arts Dojo,Bike Trail
1,Annadale,American Restaurant,Pizza Place,Bakery,Restaurant,Pharmacy,Train Station,Diner,Sushi Restaurant,Dance Studio,Pub
2,Arden Heights,Pizza Place,Pharmacy,Bus Stop,Coffee Shop,Home Service,Women's Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit
3,Arlington,Bus Stop,Intersection,Boat or Ferry,American Restaurant,Deli / Bodega,Grocery Store,Women's Store,Field,Fast Food Restaurant,Farmers Market
4,Arrochar,Bus Stop,Deli / Bodega,Italian Restaurant,Food Truck,Supermarket,Mediterranean Restaurant,Outdoors & Recreation,Middle Eastern Restaurant,Bagel Shop,Pharmacy


### 3. Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [23]:
# set number of clusters
kclusters = 5
ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
ny_merged = df
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
ny_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0.0,Caribbean Restaurant,Ice Cream Shop,Sandwich Place,Donut Shop,Gas Station,Dessert Shop,Laundromat,Pharmacy,Food,Exhibit
1,Bronx,Co-op City,40.874294,-73.829939,0.0,Bus Station,Fast Food Restaurant,Discount Store,Baseball Field,Park,Grocery Store,Bagel Shop,Pharmacy,Gift Shop,Mattress Store
2,Bronx,Eastchester,40.887556,-73.827806,0.0,Caribbean Restaurant,Diner,Deli / Bodega,Bus Station,Chinese Restaurant,Metro Station,Donut Shop,Bus Stop,Convenience Store,Juice Bar
3,Bronx,Fieldston,40.895437,-73.905643,0.0,Music Venue,River,Plaza,Food Stand,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
4,Bronx,Riverdale,40.890834,-73.912585,0.0,Bus Station,Bank,Plaza,Gym,Park,Medical Supply Store,Baseball Field,Home Service,Playground,Food Truck


#### Finally, let's visualize the resulting clusters

In [25]:
# drop row containing NaN value
ny_merged.dropna(inplace=True)

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 4. Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [28]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Caribbean Restaurant,Ice Cream Shop,Sandwich Place,Donut Shop,Gas Station,Dessert Shop,Laundromat,Pharmacy,Food,Exhibit
1,Co-op City,Bus Station,Fast Food Restaurant,Discount Store,Baseball Field,Park,Grocery Store,Bagel Shop,Pharmacy,Gift Shop,Mattress Store
2,Eastchester,Caribbean Restaurant,Diner,Deli / Bodega,Bus Station,Chinese Restaurant,Metro Station,Donut Shop,Bus Stop,Convenience Store,Juice Bar
3,Fieldston,Music Venue,River,Plaza,Food Stand,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
4,Riverdale,Bus Station,Bank,Plaza,Gym,Park,Medical Supply Store,Baseball Field,Home Service,Playground,Food Truck
5,Kingsbridge,Pizza Place,Bar,Deli / Bodega,Sandwich Place,Latin American Restaurant,Supermarket,Bakery,Mexican Restaurant,Fried Chicken Joint,Spanish Restaurant
6,Marble Hill,Coffee Shop,Sandwich Place,Yoga Studio,Diner,Tennis Stadium,Gym,Seafood Restaurant,Bank,Miscellaneous Shop,Donut Shop
7,Woodlawn,Deli / Bodega,Food Truck,Playground,Pub,Food & Drink Shop,Bar,Pizza Place,Supermarket,Park,Donut Shop
8,Norwood,Pizza Place,Bank,Park,Deli / Bodega,Bus Station,Pharmacy,Fast Food Restaurant,Caribbean Restaurant,Pet Store,Chinese Restaurant
9,Williamsbridge,Caribbean Restaurant,Bar,Nightclub,Soup Place,Field,Event Service,Event Space,Exhibit,Eye Doctor,Factory


#### Cluster 2

In [29]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,Somerville,Park,Women's Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant
203,Todt Hill,Park,Women's Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant
303,Bayswater,Playground,Park,Women's Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant


#### Cluster 3

In [30]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
179,Neponsit,Beach,Women's Store,Filipino Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm


#### Cluster 4

In [31]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
226,Graniteville,Supermarket,Women's Store,Filipino Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant


#### Cluster 5

In [32]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,Mill Island,Pool,Women's Store,Field,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory
238,Butler Manor,Baseball Field,Pool,Convenience Store,Bus Stop,Women's Store,Field,Event Service,Event Space,Exhibit,Eye Doctor
